In [35]:
Project = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte'

import pandas as pd
import numpy as np

In [36]:
RD = pd.read_csv(f'{Project}/Results/Tables/CPM_RD_calculation/v20221217/adi_RD_log2FC.tsv', sep='\t').rename(columns={'gene_name':'to'})
display(RD)

,gene_id,to,RD4a,RD4b,RD4c,RD8a,RD8b,RD8c
0,ENSMUSG00000033845,Mrpl15,-0.671003,-0.069029,-0.193291,-0.341792,-0.441117,-0.363656
1,ENSMUSG00000025903,Lypla1,-0.194914,0.463091,0.163485,-0.464949,0.161578,-0.335146
2,ENSMUSG00000033813,Tcea1,-0.008186,0.401248,0.062004,0.168080,0.165488,0.236005
3,ENSMUSG00000033793,Atp6v1h,0.063644,0.347290,0.398696,0.084042,0.363091,0.404513
4,ENSMUSG00000025907,Rb1cc1,0.165708,0.342509,-0.241816,-0.234984,-0.623727,-0.119535
...,...,...,...,...,...,...,...,...
11805,ENSMUSG00000064360,mt-Nd3,0.019150,1.072969,-1.775453,-1.965660,-2.617272,-2.638280
11806,ENSMUSG00000064363,mt-Nd4,-1.840285,-0.182261,-1.046066,-1.764191,-0.912711,-1.274588
11807,ENSMUSG00000064367,mt-Nd5,-0.629857,0.198241,-0.824479,-0.826304,-0.370965,-0.510705
11808,ENSMUSG00000064368,mt-Nd6,-0.901645,-0.145785,-0.779583,-0.715231,0.115233,-0.355842


In [37]:
func_anno = pd.read_csv(f'{Project}/Results/STRING/RD_GSEA_GOCC_lowest3terms/string_functional_annotations.tsv', sep='\t')

display(func_anno)

,#node,identifier,category,term ID,term description
0,Afg3l1,10090.ENSMUSP00000001520,GO Process,GO:0006508,Proteolysis
1,Afg3l1,10090.ENSMUSP00000001520,GO Process,GO:0006807,Nitrogen compound metabolic process
2,Afg3l1,10090.ENSMUSP00000001520,GO Process,GO:0006996,Organelle organization
3,Afg3l1,10090.ENSMUSP00000001520,GO Process,GO:0007005,Mitochondrion organization
4,Afg3l1,10090.ENSMUSP00000001520,GO Process,GO:0007006,Mitochondrial membrane organization
...,...,...,...,...,...
18836,mt-Nd6,10090.ENSMUSP00000081002,UniProt Keywords,KW-1133,Transmembrane helix
18837,mt-Nd6,10090.ENSMUSP00000081002,UniProt Keywords,KW-1278,Translocase
18838,mt-Nd6,10090.ENSMUSP00000081002,Pfam,PF00499,NADH-ubiquinone/plastoquinone oxidoreductase c...
18839,mt-Nd6,10090.ENSMUSP00000081002,InterPro,IPR001457,"NADH:ubiquinone/plastoquinone oxidoreductase, ..."


In [38]:
slct_terms = [
'Mitochondrial respiratory chain complex I',
'Respiratory chain complex II',
'Mitochondrial respiratory chain complex III',
'Respiratory chain complex IV',
'Mitochondrial proton-transporting ATP synthase complex',
'TIM22 mitochondrial import inner membrane insertion complex',
'TIM23 mitochondrial import inner membrane translocase complex',
'm-AAA complex',
'Uniplex complex',
'MICOS complex'
]

node_info = func_anno[['#node']].copy().drop_duplicates().reset_index(drop=True)

i = 1
for term in slct_terms:
    tmp = func_anno[ func_anno['term description']==term ][['#node']].drop_duplicates()
    tmp[term] = i
    node_info = pd.merge(node_info, tmp, on='#node', how='outer')
    i+=1

node_info = node_info.rename(columns={'#node':'name'}).replace(np.nan, 0).set_index('name', drop=True)
node_info.loc['Ndufa4', 'Mitochondrial respiratory chain complex I'] = 0.0
node_info.loc['mt-Co1', 'Mitochondrial respiratory chain complex III'] = 0.0
node_info['Group'] = node_info.sum(axis=1)
node_info = node_info[['Group']]
for i,term in enumerate(['None']+slct_terms):
    node_info = node_info.replace(i, term)

node_info = node_info.reset_index().rename(columns={'name':'to','Group':'from'})[['from','to']]
node_info['from'] = pd.Categorical(node_info['from'], slct_terms+['None'])
node_info = node_info.sort_values(by=['from','to'])

node_info = pd.merge(node_info, RD, on='to', how='left').drop(columns='gene_id')


display(node_info)
node_info.to_csv('node_info.tsv', sep='\t', index=False)

,from,to,RD4a,RD4b,RD4c,RD8a,RD8b,RD8c
0,Mitochondrial respiratory chain complex I,Dmac1,-0.632790,-0.082719,-0.608732,-0.202302,-0.432890,-0.409369
1,Mitochondrial respiratory chain complex I,Ndufa1,-0.840613,-0.394426,-0.235501,-0.337839,-0.809830,-0.384136
2,Mitochondrial respiratory chain complex I,Ndufa11,-0.866813,-0.200790,-0.138075,-0.452799,-1.043489,-0.616076
3,Mitochondrial respiratory chain complex I,Ndufa12,-0.943549,-0.641682,-0.791421,-0.263046,-0.937079,-0.628023
4,Mitochondrial respiratory chain complex I,Ndufa13,-0.920404,-0.502558,-0.646280,-0.377022,-1.064046,-0.851385
...,...,...,...,...,...,...,...,...
93,MICOS complex,Micos13,-0.655327,-0.168724,-0.136811,-0.037025,-0.353230,-0.208057
94,None,Higd1a,-0.413895,-0.456240,-0.156658,-0.261648,-0.530009,-0.430454
95,None,Immp2l,-0.951407,-0.930714,-0.565419,-0.361801,-1.241555,-0.956161
96,None,Phb,-0.457415,-0.120292,-0.706653,0.317011,-0.617663,-0.324291


---

In [39]:
edge_info = pd.read_csv(f'{Project}/Results/STRING/RD_GSEA_GOCC_lowest3terms/string_interactions_short.tsv', sep='\t')
display(edge_info)


edge_info = edge_info[['#node1','node2','combined_score']].rename(columns={'#node1':'from','node2':'to','combined_score':'value'})
edge_info.to_csv('edge_info.tsv', sep='\t', index=False)
display(edge_info)

,#node1,node2,node1_string_id,node2_string_id,homology,experimentally_determined_interaction,database_annotated,automated_textmining,combined_score
0,Afg3l1,Phb,10090.ENSMUSP00000001520,10090.ENSMUSP00000119603,0,0.201,0.0,0.280,0.400
1,Afg3l1,Afg3l2,10090.ENSMUSP00000001520,10090.ENSMUSP00000025408,0,0.000,0.9,0.667,0.965
2,Afg3l2,Smdt1,10090.ENSMUSP00000025408,10090.ENSMUSP00000023086,0,0.000,0.5,0.000,0.499
3,Afg3l2,Phb,10090.ENSMUSP00000025408,10090.ENSMUSP00000119603,0,0.201,0.5,0.280,0.687
4,Atp5b,Cox5a,10090.ENSMUSP00000026459,10090.ENSMUSP00000000090,0,0.299,0.0,0.266,0.463
...,...,...,...,...,...,...,...,...,...
1461,mt-Nd3,mt-Nd6,10090.ENSMUSP00000080998,10090.ENSMUSP00000081002,0,0.930,0.9,0.978,0.999
1462,mt-Nd3,mt-Nd4,10090.ENSMUSP00000080998,10090.ENSMUSP00000081000,0,0.937,0.9,0.980,0.999
1463,mt-Nd4,mt-Nd6,10090.ENSMUSP00000081000,10090.ENSMUSP00000081002,0,0.843,0.9,0.915,0.998
1464,mt-Nd4,mt-Nd5,10090.ENSMUSP00000081000,10090.ENSMUSP00000081001,0,0.959,0.9,0.980,0.999


,from,to,value
0,Afg3l1,Phb,0.400
1,Afg3l1,Afg3l2,0.965
2,Afg3l2,Smdt1,0.499
3,Afg3l2,Phb,0.687
4,Atp5b,Cox5a,0.463
...,...,...,...
1461,mt-Nd3,mt-Nd6,0.999
1462,mt-Nd3,mt-Nd4,0.999
1463,mt-Nd4,mt-Nd6,0.998
1464,mt-Nd4,mt-Nd5,0.999
